In [296]:

%matplotlib inline
import tensorflow as tf
import os
from keras.backend import set_session
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import np_utils
from keras.callbacks import EarlyStopping 
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras import optimizers

In [297]:

def create_session(gpu_id='0', pp_mem_frac=None):

    tf.reset_default_graph()
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id # can multiple?
    with tf.device('/gpu:' + gpu_id):
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        if pp_mem_frac is not None:
            config.gpu_options.per_process_gpu_memory_fraction=pp_mem_frac
        session = tf.Session(config = config)
    return session
gpu_id = '0'
sess = create_session(gpu_id)
set_session(sess)


In [298]:
#讀取檔案+整理成可丟進ann的格式
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

In [299]:
#把label改成one hot編碼
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [300]:
# Functional API
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate
a1 = Dense(200, activation='relu')     
a2 = Dense(200, activation='linear')     
a3 = Dense(100, activation='relu')    
a4 = Dense(100, activation='linear')   
a5 = Dense(20, activation='relu')   
a6 = Dense(20, activation='relu')
a7 = Dense(10, activation='softmax')  
x = Input(shape = (784,))
L11 = a1(x)
L12 = a2(x)
L21 = a3(L11)
L22 = a4(L12)
L31 = a5(L21) #concatenate1
L32 = a6(L22) #concatenate2
u = concatenate([L31,L32])
y = a7(u)

In [301]:
model = Model(x, y)
model.summary() 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 200)          157000      input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 200)          157000      input_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 100)          20100       dense_1[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

In [302]:
sgd = optimizers.SGD(lr=0.00021, decay=1e-9, momentum=0.9, clipvalue=1.)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

In [303]:
model.fit(x_train, y_train,validation_split=0.1, epochs=30,batch_size = 100)

Train on 54000 samples, validate on 6000 samples
Epoch 1/30
54000/54000 [==============================] - 2s 40us/step - loss: 7.5545 - acc: 0.5232 - val_loss: 3.2886 - val_acc: 0.7895
Epoch 2/30
54000/54000 [==============================] - 2s 35us/step - loss: 3.6740 - acc: 0.7661 - val_loss: 2.9647 - val_acc: 0.8120
Epoch 3/30
54000/54000 [==============================] - 2s 34us/step - loss: 3.1337 - acc: 0.8014 - val_loss: 2.9963 - val_acc: 0.8105
Epoch 4/30
54000/54000 [==============================] - 2s 34us/step - loss: 3.0278 - acc: 0.8081 - val_loss: 2.5353 - val_acc: 0.8395
Epoch 5/30
54000/54000 [==============================] - 2s 34us/step - loss: 2.7997 - acc: 0.8231 - val_loss: 2.3877 - val_acc: 0.8490
Epoch 6/30
54000/54000 [==============================] - 2s 34us/step - loss: 2.7677 - acc: 0.8249 - val_loss: 2.4027 - val_acc: 0.8473
Epoch 7/30
54000/54000 [==============================] - 2s 34us/step - loss: 2.5749 - acc: 0.8369 - val_loss: 1.2584 - val_acc:

In [304]:
score = model.evaluate(x_test, y_test)
print("loss", score[0])
print("accuracy", score[1])

10000/10000 [==============================] - 0s 34us/step
loss 0.7289911018445715
accuracy 0.9526
